In [1]:
import praw
import re
import pandas as pd
from datetime import datetime

# 1. Configuration de l'API Reddit
# Vous devez créer une "App" sur https://www.reddit.com/prefs/apps pour obtenir ces clés
reddit = praw.Reddit(
    client_id="26oxZPe1Jd05btZUJiEFqA",
    client_secret="K2yj5uNh1dJDryrormSGOSUejLZzoA",
    user_agent="AnalyseurVenteIntelligente v2.0"
)


In [2]:
def extract_price(text):
    """
    Tente d'extraire un prix (ex: $50, 50€, 50 USD) du texte via une expression régulière simple.
    """
    # Regex basique pour les devises courantes ($, €, Ar, etc.)
    price_pattern = r'(\$|€|Ar)\s?(\d+(?:[.,]\d{1,2})?)'
    match = re.search(price_pattern, text)
    if match:
        return match.group(0)
    return None

def scrape_reddit_products(keywords, subreddits="all", limit=50):
    """
    Recherche des produits sur Reddit basés sur des mots-clés.
    """
    print(f"--- Recherche de '{keywords}' dans r/{subreddits} ---")
    
    products_data = []
    
    # Recherche dans le subreddit spécifié (ou "all")
    subreddit = reddit.subreddit(subreddits)
    
    # On cherche les nouveaux posts correspondants à la requête
    for submission in subreddit.search(keywords, sort='hot', limit=limit):
        
        # Combiner le titre et le contenu pour l'analyse
        full_text = f"{submission.title} {submission.selftext}"
        
        # Tentative d'extraction de prix
        price = extract_price(full_text)
        
        product_info = {
            'titre': submission.title,
            'prix_estime': price, # Peut être None si pas trouvé
            'url': submission.url,
            'date_creation': datetime.fromtimestamp(submission.created_utc),
            'subreddit': submission.subreddit.display_name,
            'score': submission.score,
            'nombre_commentaires': submission.num_comments,
            'description': submission.selftext[:200] + "..." # Aperçu de la description
            
        }
        
        products_data.append(product_info)
    
    return pd.DataFrame(products_data)

# --- EXEMPLE D'UTILISATION ---

# Mots-clés pour trouver des produits (à adapter selon votre niche)
# Ex: "selling laptop", "à vendre téléphone", etc.
search_query = "selling laptop" 

# Subreddits ciblés (ex: hardwareswap, leboncoin, ou "all" pour tout reddit)
target_subreddits = "hardwareswap+appleswap" 

df_produits = scrape_reddit_products(search_query, target_subreddits, limit=50)

# Affichage des résultats
if not df_produits.empty:
    print(f"\n{len(df_produits)} produits trouvés :")
    print(df_produits[['titre', 'prix_estime', 'url']])
    
    # Export vers CSV pour la partie "Base de données" mentionnée dans votre doc
    df_produits.to_csv('produits_reddit.csv', index=False)
    print("\nDonnées sauvegardées dans 'produits_reddit.csv'")
else:
    print("Aucun produit trouvé.")

--- Recherche de 'selling laptop' dans r/hardwareswap+appleswap ---

50 produits trouvés :
                                                titre prix_estime  \
0   [USA-CA] [H] Asus Vivobook 15 inch [W] PayPal ...        $600   
1   [USA-TX] [H] zephyrus g16 4070 [W] paypal/loca...        None   
2   [USA-NY] [H] Asus Zephyrus G14 2024, Noctua NF...       $1300   
3   [USA-TX][H] 7800x3d + 9070XT Gaming PC, Ideapa...        $250   
4   [USA-IL] [H] ASUS Zephyrus G16 16" OLED 2.5K 2...       $1700   
5   [USA-CA] [H] Surface Laptop 7 Snapdragon X Eli...        None   
6   [USA-KY] [H] Alienware M18 R1 (AMD) | Retroid ...        $800   
7   [USA-KY][H] Open-Box Lenovo Yoga Pro 9 (16" 2....       $1,10   
8   [USA-CO][H] 2 in box WII consoles, cameras, To...        $130   
9   [USA-MD][H] PC Build: AMD Ryzen 5 3600, MSI B5...        $400   
10  [USA-MI][H] Several DDR5 Rams, 16gb and 128gb ...        $250   
11  [USA-NH][H] 4080 Super, Glorious Gaming Mice, ...        $875   
12  [USA-PA]